In [ ]:
import pandas as pd
import numpy as np
import warnings
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

warnings.filterwarnings("ignore")

# 1. Veriyi oku
df = pd.read_csv("/content/IOT_DATA.csv")

# 2. 60 satırı bir kişi olarak grupla
group_size = 60
num_groups = len(df) // group_size
grouped_data = []

for i in range(num_groups):
    group = df.iloc[i * group_size : (i + 1) * group_size]
    features = group[['temperature', 'humidity', 'gsr', 'pulse']].mean().to_dict()
    label = group['label'].mode()[0]
    features['label'] = label
    grouped_data.append(features)

df_grouped = pd.DataFrame(grouped_data)

# 3. Özellikleri ve hedef değişkeni ayır
X = df_grouped[['temperature', 'humidity', 'gsr', 'pulse']]
y = df_grouped['label']

# 4. Etiketleri sayısal hale getir
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 5. Eğitim ve test verilerine ayır
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.20, random_state=84, stratify=y_encoded
)

# 6. Geliştirilmiş Random Forest modeli
model = RandomForestClassifier(
    n_estimators=600,
    max_depth=80,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    class_weight='balanced',
    random_state=84
)

# 7. Modeli eğit
model.fit(X_train, y_train)

# 8. Etiketlerde "çok" → "cok" düzeltmesi
le.classes_ = np.array([label.replace("çok", "cok") for label in le.classes_])

# 9. Tahmin yap
y_pred = model.predict(X_test)

# 10. Sonuçları yazdır
accuracy = accuracy_score(y_test, y_pred)
print(f"\n📊 Improved Random Forest Accuracy: {accuracy:.4f}\n")
print("📋 Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# 11. Tahmin sonuçları
results = pd.DataFrame({
    "Gerçek": le.inverse_transform(y_test),
    "Tahmin": le.inverse_transform(y_pred)
})
print("🔍 İlk 10 Tahmin Sonucu:")
print(results.head(10))

# 12. Modeli ve LabelEncoder'ı kaydet
joblib.dump(model, "stress_rf_model.pkl")
joblib.dump(le, "label_encoder.pkl")
print("\n✅ Model ve LabelEncoder başarıyla kaydedildi.")



📊 Improved Random Forest Accuracy: 0.7778

📋 Classification Report:

              precision    recall  f1-score   support

       rahat       0.75      1.00      0.86         3
     stresli       1.00      0.33      0.50         3
 cok stresli       0.75      1.00      0.86         3

    accuracy                           0.78         9
   macro avg       0.83      0.78      0.74         9
weighted avg       0.83      0.78      0.74         9

🔍 İlk 10 Tahmin Sonucu:
        Gerçek       Tahmin
0        rahat        rahat
1  cok stresli  cok stresli
2        rahat        rahat
3      stresli  cok stresli
4  cok stresli  cok stresli
5  cok stresli  cok stresli
6        rahat        rahat
7      stresli      stresli
8      stresli        rahat

✅ Model ve LabelEncoder başarıyla kaydedildi.
